<a href="https://colab.research.google.com/github/rosangithub/ANIDS/blob/main/ANIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Anomaly Based Network Intrusion Detection System using Ensemble Machine learning


In [1]:
import numpy as np#linear algebra
import pandas as pd#data preprocessing,CSV files
import pickle#saving and loading trained model
from os import path
#importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import(StandardScaler,OrdinalEncoder,LabelEncoder,MinMaxScaler,OneHotEncoder)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer,MaxAbsScaler,RobustScaler,PowerTransformer
#importing library for plotting
import matplotlib.pyplot as plt
import seaborn as sns
import os

ModuleNotFoundError: No module named 'google.colab'

In [5]:
import glob
#path to your folder containing csv files
folder_path="/content/drive/MyDrive/data/raw/"
#get  a list of all csv files in the folder
csv_files=glob.glob(folder_path + "*.csv")
#Read all the csv files and combine them into one dataframe
df_list=(pd.read_csv(file) for file in csv_files)
df=pd.concat(df_list,ignore_index=True)
df.head()

ValueError: No objects to concatenate

In [ ]:
#display the shape and the find the concatinated dataframe
nRow,nCol=df.shape
print(f"Rows:{nRow},Columns: {nCol}")


In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.columns=df.columns.str.strip()
df['Label'].value_counts()

## Remove the missing values NaN from the dataframe

In [ ]:
df.dropna(inplace=True)

In [ ]:
df

## Plotting the histogram of the data

In [ ]:
import matplotlib.pyplot as plt  # for plotting graphs
import numpy as np               # for numerical operations and dtype checking
%matplotlib inline

def plot_per_column_distribution(df, n_graph_shown, n_graph_per_row):
    """
    This function plots the distribution of values for multiple columns in a DataFrame.
    Numeric columns are shown as histograms.
    Categorical columns (with few unique values) are shown as bar charts.

    Parameters:
    df: pandas DataFrame
        The dataset containing columns to plot
    n_graph_shown: int
        Maximum number of columns to plot
    n_graph_per_row: int
        Number of plots to show in each row
    """

    # Count unique values in each column
    nunique = df.nunique()

    # Select columns with 2 to 49 unique values for plotting
    columns_to_plot = df.columns[(nunique > 1) & (nunique < 50)]
    print(columns_to_plot)

    # Create a smaller DataFrame containing only selected columns
    df_to_plot = df[columns_to_plot]

    # Get the number of rows and columns in the filtered DataFrame
    n_rows, n_cols = df_to_plot.shape

    # Calculate how many rows of plots we need in the figure
    n_graph_row = (n_cols + n_graph_per_row - 1) // n_graph_per_row  # integer division

    # Create the figure with calculated size
    plt.figure(num=None,
               figsize=(6 * n_graph_per_row, 8 * n_graph_row),  # width x height in inches
               dpi=80,                                         # resolution
               facecolor='w',                                  # figure background color
               edgecolor='k')                                  # figure border color

    # Loop through each selected column (up to n_graph_shown)
    for i, col_name in enumerate(columns_to_plot[:n_graph_shown]):
        # Create a subplot in the grid
        plt.subplot(n_graph_row, n_graph_per_row, i + 1)

        # Get data of the current column
        column_data = df_to_plot[col_name]

        # Check if the column is numeric
        if np.issubdtype(column_data.dtype, np.number):
            # If numeric, plot histogram
            column_data.hist()
        else:
            # If categorical, count unique values and plot bar chart
            value_counts = column_data.value_counts()
            value_counts.plot.bar()

        # Label y-axis
        plt.ylabel('counts')

        # Rotate x-axis labels for better readability
        plt.xticks(rotation=90)

        # Set the title of the subplot using f-string
        plt.title(f'{col_name} (column {i})')

    # Adjust spacing between subplots to prevent overlap
    plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

    # Display the figure
    plt.show()


In [ ]:
plt.rcParams['figure.figsize']=(30,30)
plot_per_column_distribution(df, 79, 5)


## plot the bar graph


In [ ]:
def bar_graph(feature):
  df[feature].value_counts().plot(kind="bar")


In [ ]:
plt.rcParams['figure.figsize']=(15,15)
bar_graph("Label")

In [ ]:
import seaborn as sns
p=sns.distplot(a=df['Flow Duration'])
plt.show()

## plotting the piechart


In [ ]:
plt.figure(figsize=(20,20))
plt.pie(df['Label'].value_counts(),labels=df['Label'].unique(),autopct="%0.2f%%")
plt.title("pie chart distribution of normal and abnormal labels")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
#increase the figure size

plt.figure(figsize=(20,20))
#piechart distribution of normal and abnormal labels
labels=df['Label'].unique()
sizes=df['Label'].value_counts()
explode=(0.1,)*len(labels)#exploide all the slices slightly
plt.pie(sizes,labels=labels,autopct="%0.2f%%",explode=explode,startangle=90)
plt.title("pie chart distribution of normal and abnormal labels")
plt.legend(labels,loc='best')
plt.show()

In [ ]:
df.replace([np.inf,-np.inf],np.nan,inplace=True)
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)


In [ ]:
df.shape

In [ ]:
import pandas as pd
#assuming df is your datafram with the given dataset
#create a list of classes you want to keep all records for
classes_to_keep = ['DoS GoldenEye','FTP-Patator','DoS slowloris','DoS Slowhttptest',
                   'Bot','SSH-Patator','Web Attack � Brute Force','Web Attack � XSS',
                   'Infiltration','Web Attack � Sql Injection','Heartbleed']

#create  a list of classes you want to limit to 100000 records
classes_to_limit=['BENIGN','DoS Hulk','PortScan','DDoS']
#filter rows for classes to keep all records
df_keep=df[df['Label'].isin(classes_to_keep)]
#filter rows for classes to keep all records
df_limit=df[df['Label'].isin(classes_to_limit)].groupby("Label").head(50000)
#combine the filtered rows
result_df=pd.concat([df_keep,df_limit])
#shuffle the resulting dataframe to mix the classes
result_df=result_df.sample(frac=1,random_state=42).reset_index(drop=True)

In [ ]:
result_df.shape

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,20))
#pie chart distribution of normal and abnormal labels
labels=result_df['Label'].unique()
sizes=result_df['Label'].value_counts()
explode=(0.1,)*len(labels)#explode all slieces slightly
plt.pie(sizes,labels=labels,autopct="%0.2f%%",explode=explode,startangle=90)
plt.title("pie chart distribution of normal and abnormal labels")
plt.legend(labels,loc='best')
plt.show()

## Label encoding


In [ ]:
from sklearn.preprocessing import LabelEncoder

#make acopy of your dataframe
df_encoded=result_df.copy()
#create alabel encoder object
le=LabelEncoder()
#fit and transform the label column and assign the transformed values to a new column
df_encoded['Label']=le.fit_transform(result_df['Label'])
#print the mapping of original labels to encoded values
label_mapping=dict(zip(le.classes_,le.transform(le.classes_)))
print(label_mapping)
#display the modified dataframe
print("\n Encoded datarame:")
df_encoded.head()

In [ ]:
import joblib
#save the label encoder to file
encoder_file_path='label_encoder.pkl'
joblib.dump(le,encoder_file_path)

## making correlation matrix

In [ ]:
import seaborn as sns
#set the correlation threshold
correlation_threshold=0.15
#calulate the correlation matrix
corr=df_encoded.corr()
#filter column based on the correlation threshold
columns_to_include=corr.columns[abs(corr['Label'])>=correlation_threshold]
filtered_corr=corr.loc[columns_to_include,columns_to_include]
#plot the filtered correlation heatmap
fig,ax=plt.subplots(figsize=(15,15))
colormap=sns.diverging_palette(150,50,as_cmap=True)
sns.heatmap(filtered_corr,cmap=colormap,annot=True,fmt=".2f")
plt.xticks(range(len(filtered_corr.columns)),filtered_corr.columns)
plt.yticks(range(len(filtered_corr.columns)),filtered_corr.columns)
plt.show()


In [ ]:
#split your data into X and Y
X=df_encoded.drop('Label',axis=1)
y=df_encoded['Label']


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
import pandas as pd

#split the data into trainng and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

#initialize random forest classifier
rf_classifier=RandomForestClassifier(n_estimators=100,random_state=42)

#fit the model to the training data
rf_classifier.fit(X_train,y_train)

#get feature importances
feature_importances=rf_classifier.feature_importances_

#create dataframe with feature with names and their importances
feature_importance_df=pd.DataFrame({'Feature':X.columns,'Importance':feature_importances})

#sort the dataframe by importance in decending order
feature_importance_df=feature_importance_df.sort_values(by='Importance',ascending=False)

#select the top 20 features
top_features=feature_importance_df.head(20)['Feature'].tolist()

#filter your original dataframe to include only top 20 features
df_top_features=df_encoded[top_features+['Label']]

#display the dataframe with top features
df_top_features.head()


In [ ]:
import matplotlib.pyplot as plt
#create a bar plot of the top 20 features and their importances
plt.figure(figsize=(12,8))
plt.barh(top_features,feature_importance_df.head(20)['Importance'],color='red')
plt.xlabel('Importance')
plt.ylabel('Features')
plt.title('Top 20 Features by Importance')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
#assumming feature_importance-df is the datafram with feature importances
#sort the dataframe by importance in  decending order
feature_importance_df=feature_importance_df.sort_values(by='Importance',ascending=False)

#create bar plot of all the features and their importance
plt.figure(figsize=(12,8))
plt.barh(feature_importance_df['Feature'],feature_importance_df['Importance'],color="blue")
plt.xlabel('Importance')
plt.ylabel('Features')
plt.title('All Features and their Importance')
plt.show()

In [ ]:
df_top_features.info()

In [ ]:
#assumming the label is the target variable
X_top_features=df_top_features.drop(['Label'],axis=1)
y_top_features=df_top_features['Label']

#split the data into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X_top_features,y_top_features,test_size=0.2,random_state=42)

In [ ]:
X_train.info()

In [ ]:
X_test.info()

## Storing the top features

In [ ]:
X_top_features.to_csv('X_top_features.csv',index=False)
y_top_features.to_csv('y_top_features.csv',index=False)

In [ ]:
print("Infinity values in X_train:",np.any(np.isinf(X_train)))
print("NaN values in X_train:",np.any(np.isnan(X_train)))
print("Infinity values in X_test:",np.any(np.isinf(X_test)))
print("NaN values in X_test:",np.any(np.isnan(X_test)))

In [ ]:
# #from sklearn.preprocessing import StandardScaler
# scaler=StandardScaler()
# X_train_scaled=scaler.fit_transform(X_train)
# X_test_scaled=scaler.transform(X_test)

In [ ]:
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

In [ ]:

def print_all_accuracy_metric(algorithm,name):
  #computing the accuracy ,f1score,precision and recall of the model performance
  #predicting the target value from the model of the samples
  y_train_log=algorithm.predict(X_train)
  y_test_log=algorithm.predict(X_test)
  acc_train_log=metrics.accuracy_score(y_train,y_train_log)

  acc_test_log=metrics.accuracy_score(y_test,y_test_log)
  print(name,": Accuracy on training Data:{:.3f}".format(acc_train_log))
  print(name,": Accuracy on test data:{:.3f}".format(acc_test_log))
  print()

  f1_score_train_log=metrics.f1_score(y_train,y_train_log,average='macro')
  f1_score_test_log=metrics.f1_score(y_test,y_test_log,average='macro')
  print(name,": F1 score on training data:{:.3f}".format(f1_score_train_log))
  print(name,": F1 score on test data:{:.3f}".format(f1_score_test_log))
  print()

  recall_score_train_log=metrics.recall_score(y_train,y_train_log,average='macro')
  recall_score_test_log=metrics.recall_score(y_test,y_test_log,average='macro')
  print(name,": Recall score on training data:{:.3f}".format(recall_score_train_log))
  print(name,": Recall score on test data:{:.3f}".format(recall_score_test_log))
  print()

  precision_score_train_log=metrics.precision_score(y_train,y_train_log,average='macro')
  precision_score_test_log=metrics.precision_score(y_test,y_test_log,average='macro')
  print(name,": Precision score on training data:{:.3f}".format(precision_score_train_log))
  print(name,": precision socre on test dat:{:.3f}".format(precision_score_test_log))
  print()

  print("\n\n Classification Report \n\n")
  print(metrics.classification_report(y_test,y_test_log))
  print("\n\n")

# Machine Learning

 ## 1.DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
clfd=DecisionTreeClassifier(criterion='entropy',max_depth=4)
clfd.fit(X_train,y_train)

print_all_accuracy_metric(clfd,"DecisionTreeClassifier")
#save the model using joblib
joblib.dump(clfd,'DecisionTreeClassifier.joblib')

## 2. SVM(Support vector machine)

In [ ]:
# from sklearn.svm import SVC
# #create the SVM model
# svm_model=SVC(kernel='linear',C=1.0)

# #fit the model in the training data
# svm_model.fit(X_train,y_train)

# print_all_accuracy_metric(svm_model,'SVM')
# #save the model using joblib
# joblib.dump(svm_model,'SVM.joblib')

## 3. RandomForestClassifier


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf=RandomForestClassifier(random_state=42)
rf_clf.fit(X_train,y_train)
print_all_accuracy_metric(rf_clf,'RandomForestClassifier')
#save the model using joblib
joblib.dump(rf_clf,'RandomForestClassifier.joblib')

## 3.KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf=KNeighborsClassifier(n_neighbors=5)
knn_clf.fit(X_train,y_train)
# make the prediction on the test set
y_pred=knn_clf.predict(X_test)
print_all_accuracy_metric(knn_clf,'KNeighborsClassifier')

#save the model using jolib
joblib.dump(knn_clf,'KneighborsClassifier.joblib')

## 4.Naive Bayes


In [ ]:
# Naive Bayes classifier model
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
nb=GaussianNB()
#fit the model
nb.fit(X_train,y_train)
print_all_accuracy_metric(nb,'Naive Bayes')
joblib.dump(nb,'GaussianNB.joblib')

In [ ]:
pip install catboost

## 5 catboost

In [ ]:
import catboost
from catboost import CatBoostClassifier
catboost_model=CatBoostClassifier(iterations=10,learning_rate=0.1,depth=6)
catboost_model.fit(X_train,y_train)
print_all_accuracy_metric(catboost_model,'CatBoostClassifier')
joblib.dump(catboost_model,'CatBoostClassifier.joblib')

##6.Logistic regression


In [ ]:
from sklearn.linear_model import LogisticRegression
logreg_model=LogisticRegression()
logreg_model.fit(X_train,y_train)
print_all_accuracy_metric(logreg_model,'LogisticRegression')
joblib.dump(logreg_model,'LogisticRegression.joblib')

##7. Hybrid Model


In [ ]:
from sklearn.ensemble import StackingClassifier,RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
#import or define print_all_accuracy_matric function
#define the meta model
meta_model=RandomForestClassifier(n_estimators=100,random_state=42)
#Define the base model
logreg_model=LogisticRegression()
clfd=DecisionTreeClassifier(criterion='entropy',max_depth=4)
knn_clf=KNeighborsClassifier(n_neighbors=5)
catboost_model=CatBoostClassifier(iterations=10,learning_rate=0.1,depth=6)
#define the stacking ensemble model
stacked_model=StackingClassifier(
    estimators=[('DescisionTree',clfd),('LogisticRegression',logreg_model),('KNN',knn_clf),('CatBoostClassifier',catboost_model)],
    final_estimator=meta_model,
    cv=5# cross validation
)
#train the stacking ensemble model
stacked_model.fit(X_train,y_train)
#evalute the peroformance of the stacking ensemble model
print_all_accuracy_metric(stacked_model,'Hybrid Model')
#joblib.dump(stacked_model,'Hybrid Model.joblib')

In [ ]:
joblib.dump(stacked_model,'StackingEnsemble.joblib')

In [ ]:
# later, when you want to make predictions
#load the model from the file
loaded_model=joblib.load('StackingEnsemble.joblib')

# Saving data for testing

In [ ]:
print(X_top_features.iloc[0])
print("\n\noutput:",y_top_features.iloc[0])

In [ ]:
# make predictions using the loaded model
#create a new dataframe with first row
first_row_df=X_top_features.head(1)
prediction=loaded_model.predict(first_row_df)
#print the prediction
print(f"The predicted class is :{prediction[0]}")

In [ ]:
print(X_top_features.iloc[1])
print("\n\noutput:",y_top_features.iloc[1])

In [ ]:
#make prediction using the loaded model
#create a new dataframe with the first row
first_row_df=X_top_features.head(2).tail(1)
prediction=loaded_model.predict(first_row_df)
#print the prediction
print(f"The predicted class is :{prediction[0]}")

In [ ]:
first_row_df.to_csv('first_row_df.csv')

In [ ]:
first_row_df

In [ ]:
#create a new datafram with the first row
first_row_df=X_top_features.head(10)
first_row_df.to_csv("first_x_10_row_df.csv",index=False)
#create a new dataframe with the first 10 row
first_row_df=y_top_features.head(10)
first_row_df.to_csv("first_y_10_row_df.csv",index=False)

In [ ]:
#create a new datafrma with the last row
first_row_df=X_top_features.head(-10)
first_row_df.to_csv("first_x_10_df_1.csv",index=False)
#create a new datframe with the first row
first_row_df=y_top_features.head(-10)
first_row_df.to_csv("first_y_10_df_1.csv",index=False)

In [ ]:
# Assuming X_top_features and y_top_feature are your dataframes
middle_rows_X=X_top_features.loc[15:24]#select rows 16 to 25
middle_rows_y=y_top_features.loc[15:24]

#save the middle rows to CSV files
middle_rows_X.to_csv("middle_x_10_rows.csv",index=False)
middle_rows_y.to_csv("middle_y_10_rows.csv",index=False)

In [ ]:
#create a new dataframe with the first row
first_row_df=X_top_features.head(1)
first_row_df.to_csv("first_x_row_df.csv",index=False)

first_row_df=y_top_features.head(1)
first_row_df.to_csv("first_y_row_df.csv",index=False)

In [ ]:
#assumming X_top_features and y_top_features are your dataframes
last_row_X=X_top_features.tail(10)
last_row_y=y_top_features.tail(10)
#save the last rows to the csv files
last_row_X.to_csv("last_x_10_rows.csv",index=False)
last_row_y.to_csv("last_y_10_rows.csv",index=False)

# Live Testing

In [ ]:
# import joblib
import pandas as pd
#load the model from the file
model_filename='StackingEnsemble.joblib'
loaded_model=joblib.load(model_filename)
#define the order and types of features manually
feature_order=[
    'Fwd Packet Length Max',
    'Fwd Packet Length Mean',
    'Bwd Packets/s',
    'Total Length of Fwd Packets',
    'Subflow Fwd Bytes',
    'Flow Packets/s',
    'Packet Length Std',
    'Flow IAT Mean',
    'Avg Fwd Segment Size',
    'Flow IAT Max',
    'Init_Win_bytes_backward',
    'Avg Bwd Segment Size',
    'Bwd Packet Length Mean',
    'Flow Duration',
    'Bwd Packet Length Std',
    'Bwd Packet Length Max',
    'Subflow Bwd Bytes',
    'Total Length of Bwd Packets',
    'Destination Port',
    'Packet Length Variance'       ]


In [ ]:
#funtion to take user input and make prediction
def make_prediction():
    user_input = {}

    # prompt the user for input for each feature
    for column in feature_order:
        value = input(f"Enter value for {column}: ")
        # convert to float (safe for most ML models)
        user_input[column] = float(value)

    # create a dataframe with the user input
    user_data = pd.DataFrame(user_input, index=[0])

    # make predictions using the loaded model
    prediction = loaded_model.predict(user_data)

    # print the prediction
    print(f"The predicted class is: {prediction[0]}")


In [ ]:
make_prediction()